## Mathematical inverse problems

1. $Ax = b$, b is known (data) and we are looking for $A^{-1}$ to get the original $x$.
2. During the task of reconstruction we are given $b$ (the projections) and we are looking for methods to get some form of method to approximate $A^{-1}$
3. There are many problems with $A^{-1}$, it is impossible to model explicitly and the problem (1) is ill-positioned in the Hadamard sense. So we are looking for methods, which approximately/closely solve the problem of (1).

### Reconstruction methods

The most known method is the filtered-backprojection (FBP), which is based on the inverse Radon-transform as follows.

#### Filtered back-projection

#### The forward transformation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale

image = shepp_logan_phantom()
image = rescale(image, scale=0.4, mode='reflect', channel_axis=None)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)

theta = np.linspace(0.0, 180.0, max(image.shape), endpoint=False)
sinogram = radon(image, theta=theta)
dx, dy = 0.5 * 180.0 / max(image.shape), 0.5 / sinogram.shape[0]
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(
    sinogram,
    cmap=plt.cm.Greys_r,
    extent=(-dx, 180.0 + dx, -dy, sinogram.shape[0] + dy),
    aspect='auto',
)

fig.tight_layout()
plt.show()

#### The backward transformation and filters

In [ ]:
import matplotlib.pyplot as plt
from skimage.transform.radon_transform import _get_fourier_filter

filters = ['ramp', 'shepp-logan', 'cosine', 'hamming', 'hann']

for ix, f in enumerate(filters):
    response = _get_fourier_filter(2000, f)
    plt.plot(response, label=f)

plt.xlim([0, 1000])
plt.xlabel('frequency')
plt.legend()
plt.show()

In [ ]:
from skimage.transform import iradon

reconstruction_fbp = iradon(sinogram, theta=theta, filter_name='ramp')
error = reconstruction_fbp - image
print(f'FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')

imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)
ax1.set_title("Reconstruction\nFiltered back projection")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nFiltered back projection")
ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

The problem with FBP are the following:
* Results are statistically not correct for SPECT data, which is $\lambda$-Poisson
* Can not be used with attenuation maps (CT) for attenuation correction
* Good for mathematically correct estimations for calibration measurements

Tasks:
1. Specialize it for SPECT data and do the following tasks:
2. Load the SPECT projection data and plot them as they are in the dicom file
3. Reconstruct the the volume (original distribution) from the projection frames

#### Maximum-likelihood Expectation Maximization (MLEM)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#for the Shepp-Logan phantom, Radon-transform, resize...
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale, resize

import math
import time

In [ ]:
# creation of MLEM datastructures and steps to run the reconstruction
image_f = shepp_logan_phantom()
image_f = resize(image_f, (3,3) )

image_f *= 255.0/image_f.max()

#print('image_f min, max: ', np.min(image_f), np.max(image_f))

image_g = radon(image_f)

#print('image_g min, max: ', np.min(image_g), np.max(image_g))

# plt.imshow(image_g)
# plt.show()

g = image_g.copy()
f = image_f.copy()

g_row, g_col = g.shape[:]
f_row, f_col = f.shape[:]

#print('g shape: ', g_row, g_col)
#print('f shape: ', f_row, f_col)
#print('')

#1.feladat. Az A mátrix meghatározsa

gs = np.reshape(g,(g.shape[0]*g.shape[1], 1))
fs = np.reshape(f,(f.shape[0]*f.shape[1], 1))

#print('gs shape: ', gs.shape)
#print('fs shape: ', fs.shape)
#print('')

A = np.zeros((gs.shape[0], fs.shape[0]),dtype=float)

N, angles_num = g.shape[:]

angles = np.linspace(0, math.pi, angles_num) # it is recommended

COS = np.zeros((N,len(angles)))
SIN = np.zeros((N,len(angles)))

x = np.linspace(-0.5,0.5,N)

for k in range(0,N):
    for a in range(0,len(angles)):
        COS[k,a] = x[k]*math.cos(angles[a])
        SIN[k,a] = x[k]*math.sin(angles[a])

f_num = 0

for i in range(0, N):
    for j in range(0, N):
        for angle in range(0,len(angles)):
            r = COS[i,angle] + SIN[j,angle]
            index = (r + 0.5)*N
            if index >= 0 and index < N:
                ind = math.floor(index)
                if  (index - math.floor(index)) > 0.5:
                    A[ind    , f_num] = 1 - (index - math.floor(index))
                    A[ind + 1, f_num] = (index - math.floor(index))
                else:
                    index = math.floor(index)
                    A[ind, f_num] = (index - math.floor(index))
                    A[ind + 1, f_num] = 1 - (index - math.floor(index))
        f_num = f_num + 1

#print('A shape: ', A.shape)
# print('')
#
f0 = fs.copy()
f1 = f0.copy()

for i in range(0, f0.shape[0]):
    if f0[i] < 0:
        f0[i] = 0

#number of pixels
m = fs.shape[0]

#number of bins
n = gs.shape[0]

In [ ]:
# running the actual reconstruction
start = time.time()

for k in range(0,10):
    for j in range(0, fs.shape[0]):
        Sa = 0
        for i in range(0, n):
            Sa = Sa + A[i, j]

        S = 0
        for i in range(0, n):
            Saf = 0
            for j_v in range(0, m):
                Saf = Saf + A[i,j_v] * f0[j_v]
            if Saf != 0:
                S = S + gs[i] * A[i, j] / Saf
        if Sa != 0:
            f1[j] = S * f0[j] / Sa
        #print('Iteration (j):', j)
    f0 = f1
    print('+++++++++++++++++++++++++++++')
    print('Iteration: ', k)

end = time.time()
print('Elapsed time: ',(end - start)/60)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ff = np.reshape(f0,(f.shape[0], f.shape[1]))
#print(np.max(f), np.min(f))

ff *= 255.0/ff.max()

print('ff min, max: ', np.min(ff), np.max(ff))

print(f)
print('+++++++++++++++++++++++++++++++++++++++++++++')
print(ff)
#print(image_f)

ax1.imshow(int(f), cmap='gray')
ax2.imshow(int(ff), cmap='gray')

plt.show()

Tasks:
1. Try to reconstruct the volume from the SPECT projections
2. Run as many iterations as possible
3. Try to parallelize the MLEM reconstruction on multiple threads, processes, interpreters ...

#### Oredered Subsets Expectation Maximization (OSEM)

### Neural computation


#### 1. Perceptrons

##### A. Single-layer perceptron

In [ ]:
# Imports

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

Perceptron architecture with pytorch

In [ ]:
class SingleLayerNet(nn.Module):
    def __init__(self, input_size, hidden_neurons, output_size):
        super(SingleLayerNet, self).__init__()
        # Define the hidden layer with input_size input features and hidden_neurons neurons
        self.hidden_layer = nn.Linear(input_size, hidden_neurons)

        # Define the output layer with hidden_neurons input features and output_size neurons
        self.output_layer = nn.Linear(hidden_neurons, output_size)

#Define a Prediction Function
    def forward(self, x):
        # Pass the input through the hidden layer and apply the sigmoid activation function
        hidden_output = torch.sigmoid(self.hidden_layer(x))

        # Pass the hidden layer output through the output layer and apply the sigmoid activation function
        y_pred = torch.sigmoid(self.output_layer(hidden_output))

        return y_pred

In [ ]:
# create a model with one neuron
model = SingleLayerNet(1, 1, 1)

Training method of the neural network

In [ ]:
# Define the loss function (criterion)
def criterion(y_pred, y_true):
    # Binary Cross Entropy Loss
    # y_pred: predicted probabilities, y_true: true labels (0 or 1)

    # Compute the negative log likelihood loss using binary cross-entropy formula
    # (y * log(y_pred) + (1 - y) * log(1 - y_pred))
    loss = -1 * (y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))

    # Calculate the mean loss over the batch
    mean_loss = torch.mean(loss)

    return mean_loss

# Assuming 'model' is an instance of your custom neural network
# Create an optimizer (Stochastic Gradient Descent - SGD)
optimizer = optim.SGD(model.parameters(), lr=0.01)

Generation of the training data

In [ ]:
# Generate synthetic data for X ranging from -30 to 29 with a step size of 1
X = torch.arange(-30, 30, 1).view(-1, 1).type(torch.FloatTensor)

# Initialize an empty tensor Y to store the labels (target values)
Y = torch.zeros(X.shape[0])

# Assign label 1.0 to elements in Y where the corresponding X value is less than or equal to -10
Y[X[:, 0] <= -10] = 1.0

# Assign label 0.5 to elements in Y where the corresponding X value falls between -10 and 10 (exclusive)
Y[(X[:, 0] > -10) & (X[:, 0] < 10)] = 0.5

# Assign label 0 to elements in Y where the corresponding X value is greater than 10
Y[X[:, 0] > 10] = 0

Run the actual training loop

In [ ]:
# Define the training loop
epochs = 5000
cost = []  # List to store the total loss at each epoch

for epoch in range(epochs):
    total_loss = 0  # Variable to store the total loss for the current epoch
    epoch = epoch + 1  # Increment the epoch count

    for x, y in zip(X, Y):
        # Forward pass: Calculate the predicted output (yhat) using the model
        yhat = model(x)

        # Calculate the loss between the predicted output (yhat) and the actual target (y)
        loss = criterion(yhat, y)

        # Backpropagation: Compute gradients of the model parameters with respect to the loss
        loss.backward()

        # Update the model parameters using the computed gradients
        optimizer.step()

        # Zero out the gradients for the next iteration to avoid accumulation
        optimizer.zero_grad()

        # Accumulate the loss for this batch of data
        total_loss += loss.item()

    # Append the total loss for this epoch to the cost list
    cost.append(total_loss)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch} done!")  # Print status after every 1000 epochs

        # Plot the result of the function approximator
        predicted_values = model(X).detach().numpy()
        plt.plot(X.numpy(), predicted_values)  # Plot the predicted values
        plt.plot(X.numpy(), Y.numpy(), 'm')  # Plot the ground truth data (Y)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend(['Predicted', 'Ground Truth'])
        plt.title(f'Epoch {epoch} - Function Approximation')
        plt.show()

Plotting the actual loss function during training

In [ ]:
# Plot the cost (loss) over epochs
plt.plot(cost, marker='o', linestyle='-', color='b', label='Training Loss')

# Set labels and title
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy Loss')
plt.title('Training Progress - Cross Entropy Loss')

# Add grid for better readability
plt.grid(True)

# Show legend
plt.legend()

# Display the plot
plt.show()

##### B. Multi-layer perceptron

Multi-layer perceptron architecture in Pytroch with forward and backward pass

In [ ]:
class SimpleMLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = torch.randn(input_size, hidden_size, requires_grad=True)
        self.b1 = torch.randn(1, hidden_size, requires_grad=True)
        self.W2 = torch.randn(hidden_size, output_size, requires_grad=True)
        self.b2 = torch.randn(1, output_size, requires_grad=True)

    def forward(self, X):
        self.z1 = torch.matmul(X, self.W1) + self.b1
        self.a1 = torch.sigmoid(self.z1)  # Hidden layer activation
        self.z2 = torch.matmul(self.a1, self.W2) + self.b2
        self.a2 = torch.sigmoid(self.z2)  # Output layer activation
        return self.a2

    def train(self, X, y, epochs=1000, lr=0.01):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X)
            #Compute loss using (Mean Squared Error)
            loss = torch.mean((output - y) ** 2)
            losses.append(loss.item())
            #update weights
            self.backward(X, y, output, lr)
            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
        return losses



Initializing the model

In [ ]:
input_size = 2
hidden_size = 4
output_size = 1
model = SimpleMLP(input_size, hidden_size, output_size)

#Train  model and store the losses
losses = model.train(X, Y, epochs=1000, lr=0.1)

Plotting the loss function

In [ ]:
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.show()

Testing the model

In [ ]:
with torch.no_grad():
    test_output = model.forward(X_test)
    test_output = (test_output > 0.5).float()
accuracy = torch.mean((test_output == y_test).float())
print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")